## Implementation of Gibbs Sampling to approximate probability queries from a Bayesian Network.

In [3]:
import random
import copy
import numpy as np
from calculations_helper import perform_gibbs_sampling

def estimate_gibbs(iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Generate an estimate for the probability distribution for a given set of query variables and evidence values

    Args:
        iterations (int): number of samples to take before we go with the estimate
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables with their respective values

    Returns:
        np.array: estimated probability distribution for the different combinations the query variables can take on
    """
    prob_distribution = np.zeros(shape=1<<len(queries))
    
    # create list of non-evidence variables
    non_evidence_variables = [int(i) for i in network.keys() if int(i) not in evidence.keys()]
    
    current_evidence = copy.deepcopy(evidence)
    # randomly initialize all the variables
    for v in non_evidence_variables:
        current_evidence[v] = (random.random() < 0.5)

    for _ in range(iterations):
        posn = perform_gibbs_sampling(non_evidence_variables, current_evidence, network, queries)
        prob_distribution[posn] += 1.0

    return prob_distribution / iterations # normalization into a probability

In [4]:
import json

queries = [0]
evidence = {3:True,4:True}

with open('bn_test_1.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

[0.6983 0.3017]


In [5]:
import json

queries = [0, 3]
evidence = {2:True}

with open('bn_test_2.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

[0.3551 0.0559 0.1086 0.4804]


In [6]:
import json

queries = [1]
evidence = {2:False}

with open('bn_test_3.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

[0.8714 0.1286]


In [ ]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('small_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))

In [ ]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('big_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(10000, bayesian_network, queries, evidence))